<a href="https://colab.research.google.com/github/sturu1/Mon-AM/blob/master/2020_08_10_%EC%98%A4%EC%A0%84%EC%88%98%EC%97%85_ResNet_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Callable class




*    \_\_init\_\_ 에서는 상태(변수들..) 정의(노드, 레이어들 정의)
*    \_\_call\_\_ 애서는 변수들 연산자와 연결(in tf, 노드 연결성 부여)




In [24]:
class Test:
  def myPrint(self):
    print("test")

instance = Test()
instance.myPrint()

class CallableTest():
  def __init__(self):
    self.a = 10
    self.b = 20
  
  def addition(self, x1, x2):
    return x1 + x2

  def __call__(self):
    print("call test")
    c = self.addition(self.a, self.b)
    return c

instance2 = CallableTest()
print(c)

test


NameError: ignored

# 커스텀 레이어


#ResNet 구현하기
![대체 텍스트](https://t1.daumcdn.net/cfile/tistory/99340B395D3EB59027?download)


*   Residual Unit
*   Residual Layer
*   ResNet
*   3개의 클래스를 구성하여 완성해보기

# 하이퍼파라메터

In [25]:
import tensorflow as tf
import numpy as np

EPOCHS = 5

# Residual Unit 구현

![대체 텍스트](https://media.arxiv-vanity.com/render-output/3759680/x1.png)

In [26]:
class ResidualUnit(tf.keras.Model):
  def __init__(self, filter_in, filter_out, kernel_size):
    super(ResidualUnit, self).__init__()
    self.bn1 = tf.keras.layers.BatchNormalization()
    self.conv1 = tf.keras.layers.Conv2D(filter_out, kernel_size, padding = 'same')

    self.bn2 = tf.keras.layers.BatchNormalization()
    self.conv2 = tf.keras.layers.Conv2D(filter_out, kernel_size, padding = 'same')

    if filter_in == filter_out: # 입력과 출력 채널이 같다면
      self.identity = lambda x : x
    else:
      self.identity = tf.keras.layers.Conv2D(filter_out, (1, 1), padding = 'same')#(1, 1)은 자기 자신을 뜻한다.

#BN은 훈련할때, 테스트 할때 서로 달라져야함, 그래서 Faslse로 셋팅
  def call(self, x, training = False, mask = None):

    h = self.bn1(x, training = training)
    h = tf.nn.relu(h)
    h= self.conv1(h)

    h = self.bn2(x, training = training)
    h = tf.nn.relu(h)
    h= self.conv2(h)

    return self.identity(x) + h

#Residual Layer 구현


In [27]:
class ResidualLayer(tf.keras.Model):
  def __init__(self, filter_in, filters, kernel_size):
    super(ResnetLayer, self).__init__()
    self.sequence = list()
    InOutFilters = zip([filter_in] + list(filters), filters)
    # 만약 filter_in = 8, filter_out = (16,16) 일때
    #(8, 16)
    #(16, 16)
    #이런식으로 필터 개수의 쌍들이 만들어 질 것임
    for f_in, f_out in InOutFilters:
      self.sequence.append(ResidualUnit(f_in, f_out, kernel_size))

  def call(self, x, training = False, mask = None):
    for unit in self.sequence:
      x = unit(x, training = training)

# ResNet 모델 정의

![대체 텍스트](https://lh3.googleusercontent.com/proxy/wWxWznVsQShuBSzC_VaS-lDAABEESIJYSnuitV0dchEA4GarGJarJVm2ZvdGXIvVFC2HxI8sYc43IvSDw3qj1hi14Ur5pXST-HOLKQ0Hh_hZDuGu9iforEV3J0-mTXXWDvHTmzcDHAEImLrIn2Nxa4nFsptTkOvfboP5aEoFqqv0w1kg73p8gA)

In [28]:
class ResNet(tf.keras.Model):#여기 이상함
  def __init__(self):
    super(ResNet, self).__init__()

    self.conv1 = tf.keras.layers.Conv2D(8, (3, 3), padding = 'same', activation = 'relu', input_shape = (28, 28, 1))

    self.res1 = ResNetLayer(8, (16, 16), (3, 3))
    self.pool1 = tf.keras.layers.MaxPool2D((2, 2))

    self.res2 = ResNetLayer(16, (32, 32), (3, 3))
    self.pool2 = tf.keras.layers.MaxPool2D((2, 2))

    self.res3 = ResNetLayer(32, (64, 64), (3, 3))

    self.flatten = tf.keras.layers.Flatten()
    self.dense1 = tf.keras.layers.Dense(128, activation = 'relu')
    self.dense2 = tf.keras.layers.Dense(128, activation = 'softmax')

    self.Latnet = 0
    
  def call(self, x, training = False, mask = None):
    x = self.conv1(x)

    x = self.res1(x, training = training)
    x = self.pool1(x)
    x = self.res2(x, training = training)
    x = self.pool2(x)
    x = self.res3(x, training = training)

    self.Latent = self.flatten(x)
    x = self.dense(self.Latent)
    x = self.dense2(x)


# 학습, 테스트 루프 정의

In [29]:
@tf.function
def train_step(model, images, labels, loss_objet, optimizer, train_loss, train_accuracy):
  with tf.GrandientTape() as tape:
    predictions = model(images, training = True)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_varlables)

  optimizer.applu_gradients(zip(gradients, model.trainable_varlables))
  train_loss(loss)
  train_accuracy(labels, predictions)

@tf.function
def test_step(model, images, labels, loss_objet, optimizer, train_loss, train_accuracy):
  predictions = model(images, training = False)
  t_loss = loss_object(labels, predictions)
  test_loss(t_loss)
  test_accuracy(labels, predictions)

#데이터셋 준비

In [30]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train = x_train[..., tf.newaxis].astype(np.float32)
x_test = x_test[..., tf.newaxis].astype(np.float32)

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [31]:
model = ResNet()
#손실함수, 최적화알고리즘 인스턴스 생성
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()
#성능측정, 지표 인스턴스 생성
train_loss = tf.keras.mertics.Mean(name = 'train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalCrossentropy(name = 'train_accuracy')

test_loss = tf.keras.mertics.Mean(name = 'test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalCrossentropy(name = 'test_accuracy')

NameError: ignored

# 학습 루프 동작

In [ ]:
for epoch in range(EPOCHS):
  for images, labels in train_ds:
    train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy)
  
  for test_images, test_labels in test_ds:
    train_step(model, test_images, test_labels, loss_object, optimizer, test_loss, test_accuracy)

  templete = 'epoch {}, loss: {}, test loss: {}, test acc: {}'

  print(templete.format(epoch + 1, train_loss.result(), train_accuracy.result(), test_loss.result(), test_accuracy.result()))

  train_loss.result_states()
  train_accuracy.result_states()
  test_loss.result_states()
  test_accuracy.result_states()